libs

In [3]:
import numpy as np
import pandas as pd
from keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import random
import os
import tensorflow as tf

print('\u2022 Using TensorFlow Version:', tf.__version__)

• Using TensorFlow Version: 2.7.0


Preparing the data

In [5]:
base_dir = './DFU'

train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')

# Directory with our training normal/abnormal pictures
train_abnormal_dir = os.path.join(train_dir, 'abnormal')
train_normal_dir = os.path.join(train_dir, 'normal')

# Directory with our validation normal/abnormal pictures
validation_abnormal_dir = os.path.join(validation_dir, 'abnormal')
validation_normal_dir = os.path.join(validation_dir, 'normal')

Looking at the data

In [6]:
train_abnormal_fnames = os.listdir( train_abnormal_dir )
train_normal_fnames = os.listdir( train_normal_dir )

print(train_abnormal_fnames[:10])
print(train_normal_fnames[:10])

['63.jpg', '189.jpg', '77.jpg', '162.jpg', '176.jpg', '88.jpg', '348.jpg', '360.jpg', '374.jpg', '228.jpg']
['63.jpg', '189.jpg', '77.jpg', '162.jpg', '176.jpg', '88.jpg', '348.jpg', '360.jpg', '374.jpg', '228.jpg']


Printing the no.of images in each file

In [7]:
print('total training abnormal images :', len(os.listdir(      train_abnormal_dir ) ))
print('total training normal images :', len(os.listdir(      train_normal_dir ) ))

print('total validation abnormal images :', len(os.listdir( validation_abnormal_dir ) ))
print('total validation normal images :', len(os.listdir( validation_normal_dir ) ))

total training abnormal images : 401
total training normal images : 409
total validation abnormal images : 113
total validation normal images : 136


Printing the images 

In [ ]:
%matplotlib inline

import matplotlib.image as mpimg
import matplotlib.pyplot as plt

# Parameters for our graph; we'll output images in a 4x4 configuration
nrows = 4
ncols = 4

pic_index = 0 # Index for iterating over images

# Set up matplotlib fig, and size it to fit 4x4 pics
fig = plt.gcf()
fig.set_size_inches(ncols*4, nrows*4)

pic_index+=8

next_abnormal_pix = [os.path.join(train_abnormal_dir, fname) 
                for fname in train_abnormal_fnames[ pic_index-8:pic_index] 
               ]

next_normal_pix = [os.path.join(train_normal_dir, fname) 
                for fname in train_normal_fnames[ pic_index-8:pic_index]
               ]

for i, img_path in enumerate(next_abnormal_pix+next_normal_pix):
    # Set up subplot; subplot indices start at 1
    sp = plt.subplot(nrows, ncols, i + 1)
    sp.axis('Off') # Don't show axes (or gridlines)
    
    img = mpimg.imread(img_path)
    plt.imshow(img)

plt.show()

Building a small model 

In [ ]:
model = tf.keras.models.Sequential([
    # Note the input shape is the desired size of the image 512X512 with 3 bytes color
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(512, 512, 3)),
    keras.layers.BatchNormalization(),
    tf.keras.layers.GlobalAveragePooling2D(2,2),
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.GlobalAveragePooling2D(2,2), 
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'), 
    tf.keras.layers.GlobalAveragePooling2D(2,2),
    # Flatten the results to feed into a DNN
    tf.keras.layers.Flatten(), 
    # 512 neuron hidden layer
    tf.keras.layers.Dense(512, activation='relu'), 
    # Only 1 output neuron. It will contain a value from 0-1 where 0 for 1 class ('abnormal') and 1 for the other ('normal')
    tf.keras.layers.Dense(1, activation='sigmoid')  
])
model.summary()


